In [58]:
import pandas as pd
from sqlalchemy import create_engine
import os
import numpy as np

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Project-2/Sport-Salaries/')


In [70]:
### Extract CSVs into DataFrames
MLB_file = "MLB_Salaries_2017_2019.csv"
MLB_df = pd.read_csv(MLB_file)
MLB_df.head()


,NAME,TEAM,POS,SALARY,YEARS,TOTAL VALUE,AVG ANNUAL,Year
0,Clayton Kershaw,LAD,SP,33000000,7 (2014-20),215000000,30714286,2017
1,Zack Greinke,ARI,SP,31876966,6 (2016-21),206500000,34416667,2017
2,David Price,BOS,SP,30000000,7 (2016-22),217000000,31000000,2017
3,Justin Verlander,DET,SP,28000000,7 (2013-19),180000000,25714286,2017
4,Miguel Cabrera,DET,1B,28000000,10 (2014-23),292000000,29200000,2017


In [64]:
# Create a filtered dataframe from specific columns
mlb_cols = ["NAME", "POS", "TEAM", "AVG ANNUAL", "Year"]
mlb_transformed= MLB_df[mlb_cols].copy()

# Rename the column headers
mlb_transformed = mlb_transformed.rename(columns={"NAME": "player_name",
                                                          "POS": "position",
                                                          "TEAM": "team",
                                                          "AVG ANNUAL": "avg_salary",
                                                          "Year": "year"})

# Clean the data by dropping NA's, clean avg_salary, and setting the index
mlb_transformed = mlb_transformed.dropna(axis = 0, how ='any') 
mlb_transformed = mlb_transformed.sort_values('team', ascending=False).drop_duplicates(subset=['player_name', 'position', 'year'])

mlb_transformed_cond = mlb_transformed['avg_salary'] != "$NaN"
mlb_transformed = mlb_transformed[mlb_transformed_cond]

mlb_transformed.set_index("year", inplace=True)
mlb_transformed.sort_values(by=['avg_salary'])
mlb_transformed.head()


,player_name,position,team,avg_salary
year,,,,
2017,Joe Blanton,RP,WSH,4000000
2017,Stephen Drew,2B,WSH,3500000
2018,Shawn Kelley,RP,WSH,5000000
2017,Anthony Rendon,3B,WSH,5800000
2018,Adam Eaton,OF,WSH,4700000


In [17]:
#connect to postgres
connection_string = "postgres:akopova@localhost:5433/sports_db"
engine = create_engine(f'postgresql://{connection_string}')


In [65]:
#create NFL player table. Says there is an error, but table has been created.
pd.read_sql_query('CREATE TABLE mlb_player_salary(year INT, player_name VARCHAR(500), position VARCHAR(100), team VARCHAR(100), avg_salary INT, PRIMARY KEY (year, player_name, position))', con=engine).head()



ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [66]:
# Confirm table
engine.table_names()


['nfl_player_salary', 'mlb_player_salary']

In [67]:
#Load into NFL player Salary
mlb_transformed.to_sql(name='mlb_player_salary', con=engine, if_exists='append', index=True)


In [68]:
pd.read_sql_query('select * from mlb_player_salary', con=engine).head()


,year,player_name,position,team,avg_salary
0,2017,Joe Blanton,RP,WSH,4000000
1,2017,Stephen Drew,2B,WSH,3500000
2,2018,Shawn Kelley,RP,WSH,5000000
3,2017,Anthony Rendon,3B,WSH,5800000
4,2018,Adam Eaton,OF,WSH,4700000
